In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os.path
from sqlalchemy import create_engine
from matplotlib import pyplot as plt
import pickle as pk
import qgrid

pd.set_option("display.max_rows",15)
%matplotlib inline

In [2]:
conn = create_engine('sqlite:///../dataset/ijcai_cff_2017.db')

In [3]:
no_of_shops = 2000
shops = pd.DataFrame({"shop_id":np.arange(start=1, stop = (no_of_shops+1))})
shops.to_sql("shops", index = False, con = conn, if_exists='replace')

In [4]:
train_data = []
train_data_raw = pd.read_sql_query("select d.shop_id, "
                                   "d.total_view + d.total_pay as input_foot_traffic "
                                   "from train_data d "
                                   #"where shop_id = {}".format(i) 
                                   ,con = conn)


In [5]:
shape_of_network = [50,128,1]
num_of_layers_dense = 2

In [6]:
train_data = train_data_raw.dropna(how = "any")
train_data = train_data.set_index("shop_id")
qgrid.show_grid(train_data.loc[1:10])
train_data.loc[1:10]

Widget Javascript not detected.  It may not be installed or enabled properly.


,input_foot_traffic
shop_id,
1,178.0
1,233.0
1,250.0
1,234.0
1,230.0
1,237.0
1,191.0
...,...
10,61.0


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer

impute_x = Imputer()
impute_y = Imputer()

scaler = StandardScaler()

def scale(x):
    return scaler.fit_transform(x)

def reverse_scale(x):
    return scaler.inverse_transform(x)

def split(x,y_):
    x_train, x_valid, y_train, y_valid = train_test_split(
    x, y_ , test_size=0.45, random_state=42)
    
    x_valid, x_test, y_valid, y_test = train_test_split(
    x_valid, y_valid , test_size=0.15, random_state=42)
    
    return x_train, x_valid, x_test, y_train, y_valid, y_test 

In [8]:
train = train_data.loc[1] # Select which shop
shape = train.values.shape
train_scaled = scale(train.values.ravel().reshape(-1, 1)).reshape(shape)
train_scaled.shape

(130, 1)

In [9]:
skip = train_scaled.shape[0] % (shape_of_network[0] ) #+ shape_of_network[-1]
if skip is not 0:
    train_trim = train_scaled[skip:,]
train_trim.shape

(100, 1)

In [10]:
step = shape_of_network[0]
x_arr = []
y_arr = []
for i in range(0, train_trim.shape[0]-skip, step):
    x_arr.append(train_trim[i:i+shape_of_network[0]].flatten())
    y_arr.append(train_trim[i+shape_of_network[0]].flatten())

x_arr = np.array(x_arr)
y_arr = np.array(y_arr)

print("x: {}".format(x_arr))
print("y: {}".format(y_arr))

IndexError: index 100 is out of bounds for axis 0 with size 100

In [ ]:
plt.plot(train.values.ravel())

In [ ]:
plt.scatter([range(x_arr[0].ravel().shape[0])],x_arr[0].ravel(), label = "x data")
plt.scatter([range(y_arr[0].ravel().shape[0])], y_arr[0].ravel(), label = "y data")
plt.legend()

In [ ]:
plt.scatter([range(x_arr[1].ravel().shape[0])],x_arr[1].ravel(), label = "x data")
plt.scatter([range(y_arr[1].ravel().shape[0])], y_arr[1].ravel(), label = "y data")
plt.legend()

In [ ]:
print("x shape {}".format(x_arr.shape))
print("y shape {}".format(y_arr.shape))
x_train, x_valid, x_test, y_train, y_valid, y_test = split(x_arr, y_arr)

In [ ]:
import tensorflow as tf

In [ ]:
graph_dense = tf.Graph()
with graph_dense.as_default():
    with tf.name_scope("dense_inputs"):
        x = tf.placeholder(dtype = tf.float64, shape = [None, shape_of_network[0]], name="x")
        y_ = tf.placeholder(dtype = tf.float64, shape = [None, shape_of_network[-1]], name = "y_")

    y_h = x
    with tf.name_scope("dense_hidden"):
        w_h = tf.Variable(tf.truncated_normal(shape = [shape_of_network[0], shape_of_network[1]], dtype = tf.float64), name = "hidden_weights")
        tf.summary.histogram(name="hidden_weights_0", values = w_h)
            
        b_h = tf.Variable(tf.constant(value = 0.1, dtype = tf.float64, shape=[shape_of_network[1]]), name="hidden_bias")
        y_h = tf.nn.relu(tf.matmul(y_h , w_h) + b_h, name="relu")
            
        for i in range(1, num_of_layers_dense):
            w_h = tf.Variable(tf.truncated_normal(shape = [shape_of_network[1], shape_of_network[1]], dtype = tf.float64), name = "hidden_weights")
            tf.summary.histogram(name="hidden_weights_{}".format(i), values = w_h)
            
            b_h = tf.Variable(tf.constant(value = 0.1, dtype = tf.float64, shape=[shape_of_network[1]]), name="hidden_bias")
        
            y_h = tf.nn.relu(tf.matmul(y_h , w_h) + b_h, name="relu")

    with tf.name_scope("dense_outputs"):
        w_o = tf.Variable(tf.truncated_normal(shape = [shape_of_network[1], shape_of_network[-1]], dtype = tf.float64), name = "output_weights")
        tf.summary.histogram(name="output_weights", values = w_o)
        b_o = tf.Variable(tf.constant(value = 0.1, dtype = tf.float64, shape=[shape_of_network[-1]]), name="output_bias")
    
        y = tf.matmul(y_h, w_o) + b_o

    with tf.name_scope("cost"):
        cost = tf.reduce_mean(
            tf.abs(
                tf.divide(
                    tf.subtract(y, y_),
                    tf.add(y, y_)
                )
            )
        , name = "cost")
        tf.summary.scalar(name="cost",  tensor = cost)

    with tf.name_scope("optimizer"):
        learning_rate=0.001
        grad_clip=5
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))

num_of_layers_lstm = 2
rnn_size = 2
batch_size = 128

graph_lstm = tf.Graph()
with graph_lstm.as_default():
    with tf.name_scope("lstm_inputs"):
        lstm_x = tf.placeholder(dtype = tf.float64, shape = [None, shape_of_network[0]], name="x")
        lstm_y_ = tf.placeholder(dtype = tf.float64, shape = [None, shape_of_network[1]], name = "y_")
        lstm_keep_prob = tf.placeholder(dtype = tf.float32)
        
    with tf.name_scope("lstm_cells"):
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

        cell = tf.contrib.rnn.MultiRNNCell([drop] * num_of_layers_lstm)
        
        initial_state = cell.zero_state(batch_size, tf.float32)
        initial_state = tf.identity(initial_state,name="initial_state")
    
    with tf.name_scope("lstm_rnn"):
        outputs, state = tf.nn.dynamic_rnn(cell, x, dtype = tf.float32)
        final_state = state
        final_state = tf.identity(final_state, "final_state")


In [ ]:
with tf.Session(graph = graph_dense) as session:
    session.run(tf.global_variables_initializer())
    merged = tf.summary.merge_all()
    file_writer_train = tf.summary.FileWriter("./logs/ijcai_2017/low_level_api/train", session.graph)
    file_writer_validation = tf.summary.FileWriter("./logs/ijcai_2017/low_level_api/validation")


    for i in range(5000):
        _, train_loss, train_summary = session.run([optimizer, cost, merged], feed_dict = {x:x_train, y_:y_train})
        valid_loss, valid_summary = session.run([cost, merged],feed_dict = {x:x_valid, y_:y_valid})

        if(i % 500 == 0):
            print("epoch: {:>4} training {:5.4f} validation {:5.4f}".format(i, train_loss, valid_loss))
            file_writer_train.add_summary(train_summary, i)
            file_writer_validation.add_summary(valid_summary, i)
        